In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies.head()

In [4]:
credits.head()

In [5]:
#credits.head(1)['cast'].values

### Merge both the datasets

In [6]:
movies = movies.merge(credits, on = 'title')

In [7]:
movies.shape

In [8]:
movies.head(1)

In [9]:
# genres
# id
# keywords
# title
# overview
# cast
#crew

movies= movies[['id','genres','keywords','title','overview','cast','crew']]
movies

In [10]:
movies.isnull().sum()

In [11]:
movies.dropna(inplace=True)

In [12]:
movies.duplicated().sum()

In [13]:
movies.iloc[0].genres

In [14]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i["name"])
    return L

In [15]:
movies['genres']=movies['genres'].apply(convert)

In [16]:
movies.head()

In [17]:
movies['keywords']=movies['keywords'].apply(convert)

In [18]:
movies.head()

In [19]:
def convert3(obj):
    L= []
    counter = 0;
    for i in ast.literal_eval(obj):
        if counter<3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L        

In [20]:
movies['cast'] = movies['cast'].apply(convert3)

In [21]:
movies.head()

In [22]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L       
        

In [23]:
movies['crew']=movies['crew'].apply(fetch_director)

In [24]:
movies.head()

In [25]:
movies['overview'][0]

In [26]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [27]:
movies.head()

In [28]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [29]:
movies.head()

In [30]:
movies['tag'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [31]:
movies.head()

In [32]:
new_df = movies[['id','title','tag']]

In [33]:
new_df

In [34]:
new_df['tag']=new_df['tag'].apply(lambda x: " ".join(x))

In [35]:
new_df.head()

In [36]:
new_df['tag'][0]

In [37]:
new_df['tag']= new_df['tag'].apply(lambda x:x.lower())

In [38]:
new_df['tag'][0]

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [40]:
vector = cv.fit_transform(new_df['tag']).toarray()

In [41]:
vector[0]

In [42]:
cv.get_feature_names()  # the string which you get by joining this tags is called corpus

In [ ]:
!pip install nltk

In [ ]:
import nltk # natural language processing library 

In [ ]:
from nltk.stem.porter import PorterStemmer  #it stems the word to its basic form 
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i)) #gives output as list
        
    return " ".join(y)                # for converting the list into string again   

In [ ]:
stem('danced')

In [ ]:
new_df['tag']=new_df['tag'].apply(stem)

In [ ]:
new_df['tag'][0]

In [ ]:
new_df['tag'][1]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


#this calculates the angle between two vectors and if the angle is less than the two vectors(movies) are more likely
# to be near to each other 

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
similarity[0]

# here similarity of forst movie with first is always 1 
# it will always be a diagonally containing 1 matrix

In [ ]:
#sorted(similarity[0]) # this will sort the indexex but we will not get exactly serial order of first movie is compared 
                      # with which movie , i.e indexex will get mixed previously it was 1st with 1st, 1st with 2nd and so on

#enumerate(similarity[0]) # converts list into tuple
 
#sorted(list(enumerate(similarity[0])),reverse = True)  # this sorts on basis of first number
    
sorted(list(enumerate(similarity[0])),reverse = True,key = lambda x:x[1])



In [ ]:
new_df[new_df['title']=='Avatar'].index[0]       # this is the way of getting index 

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0] 
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse = True,key = lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]]['title'])
        
    

In [ ]:
new_df.iloc[1216]['title']  # this will give you th title of the movie

In [ ]:
recommend('Avatar')

In [ ]:
recommend("Pirates of the Caribbean: At World's End")

In [ ]:
recommend("Spectre")

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
movie_id = movies.iloc[new_df[0]].movie_id